In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import datetime
import time
import urllib
import json

app_id = "443809049300463"
app_secret = "e6ff2a431bb3da7624faefbf39a15a3d" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret
access_token

test_post_url = pd.read_csv('1717731545171536_facebook_statuses.csv')["b'post_id'"][0]

In [2]:
def request_until_succeed(url):
    req = urllib.request.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib.request.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
            
            print("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return response.read()

In [3]:
client = MongoClient()
db = client.meme_scrape
# for i in range(2):
#     x = parseAllReactions(pd.read_csv('1717731545171536_facebook_statuses.csv')["b'post_id'"][i], 100)
#     end = db.reactions_full.insert_one(x)
db.user_reactions.drop()

In [33]:
def parseAllReactions(post_id, num_reactions):
    base = "https://graph.facebook.com/v2.8/"+post_id+"/reactions"
    parameters = "?access_token=%s&limit=%s" % (access_token, num_reactions)
    
    url = base + parameters
    
    scrape_starttime = datetime.datetime.now()
    data = json.loads(request_until_succeed(url).decode('utf-8'))
    
    post_data = []
    pages = 0
    has_next_page = True
    
    while has_next_page:
    #    post_data = post_data + data['data']
        for i in data['data']:
            db.user_reactions.update_one( { "_id": i['id'] } , 
                                        { "$push": { 'reactions' : {'type' : i['type'], 'post_id' : post_id} } }, upsert = True)
        pages+=1
        # if there is no next page, we're done.
        if 'next' in data['paging'].keys():
            data = json.loads(request_until_succeed(data['paging']['next']).decode('utf-8'))
        else:
            has_next_page = False
            
    #db.post_reactions.insert_one( {'_id' : post_id, 'reactions' : post_data} )
    print("\nDone!\n%s pages Processed in %s" % (pages, datetime.datetime.now() - scrape_starttime))


In [32]:
# clear everything cell
db.user_reactions.drop()
db.post_reactions.drop()

In [34]:
for i in range(30):
    parseAllReactions(pd.read_csv('1717731545171536_facebook_statuses.csv')["b'post_id'"][i], 100)


Done!
56 pages Processed in 0:00:11.163093

Done!
19 pages Processed in 0:00:04.400053

Done!
11 pages Processed in 0:00:03.624360

Done!
31 pages Processed in 0:00:05.723621

Done!
10 pages Processed in 0:00:01.842816

Done!
29 pages Processed in 0:00:06.325942

Done!
9 pages Processed in 0:00:02.152977

Done!
25 pages Processed in 0:00:05.409622

Done!
12 pages Processed in 0:00:02.868095

Done!
56 pages Processed in 0:00:12.298252

Done!
20 pages Processed in 0:00:04.041785

Done!
9 pages Processed in 0:00:02.087123

Done!
7 pages Processed in 0:00:01.692024

Done!
84 pages Processed in 0:00:20.109329

Done!
12 pages Processed in 0:00:02.744283

Done!
32 pages Processed in 0:00:06.698033

Done!
35 pages Processed in 0:00:07.319467

Done!
5 pages Processed in 0:00:00.968229

Done!
25 pages Processed in 0:00:04.798996

Done!
16 pages Processed in 0:00:03.452977

Done!
54 pages Processed in 0:00:12.245529

Done!
36 pages Processed in 0:00:07.479278

Done!
8 pages Processed in 0:00:01.

In [36]:
# Gets the reaction breakdown for the first 30 posts
s = db.post_reactions.find()
for i in s:
    l=np.array([])
    for k in i['reactions']:
        l = np.append(l,k['type'])
    num_like = np.count_nonzero(l=='LIKE')
    num_love = np.count_nonzero(l=='LOVE')
    num_haha = np.count_nonzero(l=='HAHA')
    num_wow = np.count_nonzero(l=='WOW')
    num_angry = np.count_nonzero(l=='ANGRY')
    num_sad = np.count_nonzero(l=='SAD')
    print(num_like, num_love, num_haha, num_wow, num_angry, num_sad)

In [35]:
# a = db.user_reactions.find()
# for i in a:
#     print(i)

In [47]:
# going to get summary breakdown for a specific user
s = db.user_reactions.find()

In [82]:
test_user_id = "436073616742711"
s = db.user_reactions.find({"_id" : test_user_id})
out = s.next()
out

{'_id': '436073616742711',
 'posts_liked': ['1717731545171536_1899685076976181',
  '1717731545171536_1899735876971101',
  '1717731545171536_1898955040382518',
  '1717731545171536_1899674983643857'],
 'reaction_breakdown': ['ANGRY', 'SAD', 'ANGRY', 'LIKE'],
 'reactions': [{'post_id': '1717731545171536_1899685076976181',
   'type': 'ANGRY'},
  {'post_id': '1717731545171536_1899735876971101', 'type': 'SAD'},
  {'post_id': '1717731545171536_1898955040382518', 'type': 'ANGRY'},
  {'post_id': '1717731545171536_1899674983643857', 'type': 'LIKE'}]}

In [51]:
out['reactions']

[{'post_id': '1717731545171536_1899685076976181', 'type': 'ANGRY'},
 {'post_id': '1717731545171536_1899735876971101', 'type': 'SAD'},
 {'post_id': '1717731545171536_1898955040382518', 'type': 'ANGRY'},
 {'post_id': '1717731545171536_1899674983643857', 'type': 'LIKE'}]

In [86]:
def formatUserReactions(user_id):
    """
    After parsing with parseAllReactions, reformats the reactions
    of a single user into a more readable format.
    Returns two things--a list of posts they've reacted to and
    their reactions to those posts.
    """
    
    user = db.user_reactions.find({"_id" : user_id}).next()
    if user == None:
        print("No reaction data for user:", user_id)
        return
    
    posts = []
    reactions = []
    for i in user['reactions']:
        posts = posts + [i['post_id']]
        reactions = reactions + [i['type']]
    
    
    
    db.user_reactions.update_one( { "_id": user_id } , 
                    { "$set": { 'posts_liked' : posts , 'reaction_breakdown' : reactions } }, upsert = True)